In [1]:
import gradio as gr
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model

In [2]:
# Preprocess function for VGG16, this function pre-processes the images for VGG_16 model
def preprocess_for_VGG_(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize for VGG16
    frame = frame / 255.0  # Normalize
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame

In [3]:
# Preprocess function for InceptionV3, this function pre-processes the images for InceptionV3 model
def preprocess_for_inception(frame):
    frame = cv2.resize(frame, (299, 299))  # Resize for InceptionV3
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    frame = preprocess_input(frame)  # Preprocess for InceptionV3
    return frame

In [4]:
# Split video into frames function
def split_video_into_frames(video_path, output_folder):
    video = cv2.VideoCapture(video_path)
    fps = int(video.get(cv2.CAP_PROP_FPS))
    interval = fps
    os.makedirs(output_folder, exist_ok=True)

    frame_count = 0
    extracted_frames = []

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        if frame_count % interval == 0:
            extracted_frames.append(frame)  # Save frame to list
        frame_count += 1

    video.release()
    return extracted_frames

In [5]:
# Get predictions for frames
def get_predictions_for_frames(frames, model_Vgg16, model_InceptionV3, weight_a=0.6, weight_b=0.4):
    predictions = []
    combined_probabilities = []

    for frame in frames:
        # Preprocess each frame for both models
        vgg_frame = preprocess_for_VGG_(frame)
        inception_frame = preprocess_for_inception(frame)

        # Get predictions from both models
        VGG_predictions = model_Vgg16.predict(vgg_frame)
        Inception_predictions = model_InceptionV3.predict(inception_frame)

        # Combine predictions with weights
        combined_prediction = (weight_a * VGG_predictions) + (weight_b * Inception_predictions)
        combined_probabilities.append(combined_prediction)
        final_prediction = np.argmax(combined_prediction)  # Multi-class classification
        predictions.append(final_prediction)

    # Calculate the average probabilities across all frames
    avg_probabilities = np.mean(combined_probabilities, axis=0)
    avg_final_prediction = np.argmax(avg_probabilities)  # Multi-class classification

    return predictions, avg_final_prediction


In [6]:
def process_input(input_file, model_Vgg16, model_InceptionV3):
    # Gradio provides file paths as strings or dictionaries
    if isinstance(input_file, dict) and "name" in input_file:
        file_path = input_file["name"]
    elif isinstance(input_file, str):
        file_path = input_file
    else:
        raise ValueError("Unsupported input_file format.")

    # Determine if the input is an image or video based on file extension
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        print("Processing an image file.")

        # Read and preprocess the image
        img = cv2.imread(file_path)
        vgg_frame = preprocess_for_VGG_(img)
        inception_frame = preprocess_for_inception(img)

        # Get predictions from both models
        VGG_predictions = model_Vgg16.predict(vgg_frame)
        print(VGG_predictions)
        Inception_predictions = model_InceptionV3.predict(inception_frame)
        print(Inception_predictions)

        # Combine predictions
        weight_a = 0.6
        weight_b = 0.4
        combined_predictions = (weight_a * VGG_predictions) + (weight_b * Inception_predictions)
        final_prediction = np.argmax(combined_predictions)

        # Map the prediction to "Fake" or "Real"
        result = "Real" if final_prediction == 1 else "Fake"
        return f"Predicted Class for Image: {result}"

    elif file_path.lower().endswith(('.mp4', '.avi', '.mov')):
        print("Processing a video file.")

        # Extract frames from video
        output_folder = "temp_frames"
        frames = split_video_into_frames(file_path, output_folder)

        # Get predictions for all frames
        predictions, avg_prediction = get_predictions_for_frames(frames, model_Vgg16, model_InceptionV3)

        # Map predictions to "Fake" or "Real"
        frame_results = ["Real" if pred == 1 else "Fake" for pred in predictions]
        avg_result = "Real" if avg_prediction == 1 else "Fake"

        return (f"Predicted Classes for Video Frames: {frame_results}\n"
                f"Averaged Predicted Class for Video: {avg_result}")

    else:
        return "Error: Unsupported file type."


In [7]:
# Load pre-trained models
model_Vgg16 = load_model(r"C:\Users\ksaks\Downloads\DATA\Models\model_epoch_37-val_loss_0.5412.h5")
model_InceptionV3 = load_model(r"C:\Users\ksaks\Downloads\DATA\Models\Model_Inception\Inception_net_epoch_50_loss_0.5707.h5")    # Get predictions from both models
  

In [8]:
# Gradio Interface
interface = gr.Interface(
    fn=lambda input_file: process_input(input_file, model_Vgg16, model_InceptionV3),
    inputs=gr.File(label="Upload an Image or Video"),
    outputs=gr.Textbox(label="Prediction Result"),
    title="Image/Video Classification",
    description="Upload an image or video. The system will classify the image or extract frames from the video and classify each frame."
)

# Launch Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Processing an image file.
1/1 [==============================] - 2s 2s/step
[[0.00767984]]
1/1 [==============================] - 3s 3s/step
[[0.36978087]]
